In [1]:
import pandas as pd
import numpy as np
from custom_function import DicodingProject1 as DP1
from tensorflow.keras.models import load_model  # type: ignore
from sklearn.metrics import accuracy_score

2025-03-25 15:04:57.737123: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-25 15:04:57.751884: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742889897.769797   72685 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742889897.774381   72685 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1742889897.785570   72685 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

#### Explore Dataset Labelled using Hartmann DistilRoberta-base

In [2]:
DATASET = 'hartmann'
df_poem = pd.read_csv(f'./dataset/labelled_poem_{DATASET}.csv')
df_poem['label'].value_counts()

label
sadness     6431
fear        3920
anger       2833
joy         2550
neutral     1025
surprise     544
disgust      109
Name: count, dtype: int64

In [ ]:
# Treshold digunakan untuk memfilter dari score yang lebih rendah (Baca ATTENTION pada file labelling.ipynb)
threshold = np.quantile(df_poem['score'], 0.4)
df_poem_filtered = df_poem[df_poem['score']>=threshold]
f'Set threshold {round(threshold, 5)}', df_poem_filtered['label'].value_counts()

('Set threshold 0.75023',
 label
 sadness     4352
 fear        2637
 anger       1608
 joy         1457
 neutral      242
 surprise     127
 disgust       24
 Name: count, dtype: int64)

In [ ]:
df_poem_filtered = df_poem_filtered[(df_poem_filtered['label']!='neutral') & (df_poem_filtered['label']!='disgust') & (df_poem_filtered['label']!='surprise')]
df_poem_filtered['label'].value_counts(), f'Banyak data adalah {df_poem_filtered['label'].value_counts().sum()}'
# Banyak data sudah lebih dari 10.000. Amann

(label
 sadness    4352
 fear       2637
 anger      1608
 joy        1457
 Name: count, dtype: int64,
 'Banyak data adalah 10054')

From 7 labels, only used 4 because the imbalance class distribution are too big

In [5]:
TEST_SIZES = [0.2, 0.1]
EPOCHS = 100
MODES = ['lstm', 'gru']
BATCH_SIZE = 64
CLASS = df_poem_filtered['label'].nunique()

for TEST_SIZE in TEST_SIZES:
    for MODE in MODES:
        print('')
        kelas = DP1(TEST_SIZE, EPOCHS, MODE, CLASS, DATASET)
        X_train_HF, X_val_HF, y_train_HF, y_val_HF = kelas.prepareDf(df_poem_filtered)
        X_train_HF_keras, X_val_HF_keras, word_counts = kelas.kerasTokenizer(X_train_HF, X_val_HF)
        train_dataset_HF = kelas.createDataset(X_train_HF_keras, y_train_HF)
        val_dataset_HF = kelas.createDataset(X_val_HF_keras, y_val_HF)
        model = kelas.buildModel(MODE, int(word_counts//4))
        model.fit(
            train_dataset_HF,
            epochs = EPOCHS,
            batch_size = BATCH_SIZE,
            validation_data = val_dataset_HF,
            callbacks=[kelas.checkpointCallback(), kelas.earlyStopping()]
        )
        best_model = load_model(f'./model/best_model_{DATASET}_{TEST_SIZE}_{MODE}.keras')
        result = best_model.predict(X_val_HF_keras)
        predicted_labels = np.argmax(result, axis=1)
        accuracy = accuracy_score(y_val_HF, predicted_labels)
        print(f'Akurasi untuk skema {DATASET}, split test {TEST_SIZE*100}%, model {MODE} adalah {accuracy*100}%')
        with open('4.1. log_testing_result.txt', 'a') as f:
            f.write(f'Akurasi untuk skema {DATASET}, split test {TEST_SIZE*100}%, model {MODE} adalah {accuracy*100}% \n')

I0000 00:00:1740039498.170495    1549 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5563 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


Epoch 1/100


I0000 00:00:1740039501.490197    1760 cuda_dnn.cc:529] Loaded cuDNN version 90300


217/218 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 0.2964 - loss: 1.3764
Epoch 1: val_accuracy improved from -inf to 0.49770, saving model to ./model/best_model_hartmann_0.2_lstm.keras
218/218 ━━━━━━━━━━━━━━━━━━━━ 19s 72ms/step - accuracy: 0.2969 - loss: 1.3762 - val_accuracy: 0.4977 - val_loss: 1.2722
Epoch 2/100
218/218 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.5621 - loss: 1.1233
Epoch 2: val_accuracy improved from 0.49770 to 0.67605, saving model to ./model/best_model_hartmann_0.2_lstm.keras
218/218 ━━━━━━━━━━━━━━━━━━━━ 18s 80ms/step - accuracy: 0.5623 - loss: 1.1228 - val_accuracy: 0.6760 - val_loss: 0.8379
Epoch 3/100
218/218 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - accuracy: 0.7533 - loss: 0.6803
Epoch 3: val_accuracy improved from 0.67605 to 0.76020, saving model to ./model/best_model_hartmann_0.2_lstm.keras
218/218 ━━━━━━━━━━━━━━━━━━━━ 21s 95ms/step - accuracy: 0.7534 - loss: 0.6800 - val_accuracy: 0.7602 - val_loss: 0.6496
Epoch 4/100
218/218 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/

---

#### Explore Dataset Labelled using Savani Distilbert-base

In [6]:
DATASET = 'savani'
df_poem = pd.read_csv(f'./dataset/labelled_poem_{DATASET}.csv')
df_poem['label'].value_counts()

label
joy         6524
sadness     4811
anger       3297
fear        1527
love        1179
surprise      74
Name: count, dtype: int64

In [7]:
threshold = np.quantile(df_poem['score'], 0.32)
df_poem_filtered = df_poem[df_poem['score']>=threshold]
f'Set threshold {round(threshold, 5)}', df_poem_filtered['label'].value_counts()

('Set threshold 0.6674',
 label
 joy         4841
 sadness     3331
 anger       1873
 fear         928
 love         813
 surprise      54
 Name: count, dtype: int64)

In [ ]:
df_poem_filtered = df_poem_filtered[(df_poem_filtered['label']!='fear') & (df_poem_filtered['label']!='disgust') & (df_poem_filtered['label']!='love') & (df_poem_filtered['label']!='surprise')]
df_poem_filtered['label'].value_counts(), f'Banyak data adalah {df_poem_filtered['label'].value_counts().sum()}'
# Banyak data sudah lebih dari 10.000. Amann

(label
 joy        4841
 sadness    3331
 anger      1873
 Name: count, dtype: int64,
 'Banyak data adalah 10045')

From 6 labels, only used 3 because the imbalance class distribution are too big

In [9]:
TEST_SIZES = [0.2, 0.1]
EPOCHS = 100
MODES = ['gru', 'lstm']
BATCH_SIZE = 64
CLASS = df_poem_filtered['label'].nunique()

for TEST_SIZE in TEST_SIZES:
    for MODE in MODES:
        print(f'Skenario Dataset {DATASET} : {TEST_SIZE} mode {MODE}')
        kelas = DP1(TEST_SIZE, EPOCHS, MODE, CLASS, DATASET)
        X_train_HF, X_val_HF, y_train_HF, y_val_HF = kelas.prepareDf(df_poem_filtered)
        X_train_HF_keras, X_val_HF_keras, word_counts = kelas.kerasTokenizer(X_train_HF, X_val_HF)
        train_dataset_HF = kelas.createDataset(X_train_HF_keras, y_train_HF)
        val_dataset_HF = kelas.createDataset(X_val_HF_keras, y_val_HF)
        model = kelas.buildModel(MODE, int(word_counts//4))
        model.fit(
            train_dataset_HF,
            epochs = EPOCHS,
            batch_size = BATCH_SIZE,
            validation_data = val_dataset_HF,
            callbacks=[kelas.checkpointCallback(), kelas.earlyStopping()]
        )
        best_model = load_model(f'./model/best_model_{DATASET}_{TEST_SIZE}_{MODE}.keras')
        result = best_model.predict(X_val_HF_keras)
        predicted_labels = np.argmax(result, axis=1)
        accuracy = accuracy_score(y_val_HF, predicted_labels)
        print(f'Akurasi untuk skema {DATASET}, split test {TEST_SIZE*100}%, model {MODE} adalah {accuracy*100}%')
        with open('4.1. log_testing_result.txt', 'a') as f:
            f.write(f'Akurasi untuk skema {DATASET}, split test {TEST_SIZE*100}%, model {MODE} adalah {accuracy*100}% \n')

Skenario Dataset savani : 0.2 mode gru
Epoch 1/100
182/182 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - accuracy: 0.3613 - loss: 1.0960
Epoch 1: val_accuracy improved from -inf to 0.50430, saving model to ./model/best_model_savani_0.2_gru.keras
182/182 ━━━━━━━━━━━━━━━━━━━━ 14s 67ms/step - accuracy: 0.3615 - loss: 1.0960 - val_accuracy: 0.5043 - val_loss: 1.0787
Epoch 2/100
181/182 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.5791 - loss: 1.0468
Epoch 2: val_accuracy improved from 0.50430 to 0.56007, saving model to ./model/best_model_savani_0.2_gru.keras
182/182 ━━━━━━━━━━━━━━━━━━━━ 14s 78ms/step - accuracy: 0.5792 - loss: 1.0465 - val_accuracy: 0.5601 - val_loss: 0.9779
Epoch 3/100
182/182 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 0.6745 - loss: 0.8293
Epoch 3: val_accuracy improved from 0.56007 to 0.70499, saving model to ./model/best_model_savani_0.2_gru.keras
182/182 ━━━━━━━━━━━━━━━━━━━━ 11s 61ms/step - accuracy: 0.6747 - loss: 0.8289 - val_accuracy: 0.7050 - val_loss: 0.7432
Epoch 4

#### Explore Dataset Labelled using Deepseek Model

In [2]:
DATASET = 'deepseek_clean'
df_poem = pd.read_csv(f'./dataset/labelled_poem_{DATASET}.csv')
df_poem['label'].value_counts()

label
other      5687
sadness    4113
joy        3733
hope       2605
love       1274
Name: count, dtype: int64

In [3]:
df_poem_filtered = df_poem[df_poem['label'] != 'other']
df_poem_filtered['label'].value_counts()

label
sadness    4113
joy        3733
hope       2605
love       1274
Name: count, dtype: int64

In [4]:
TEST_SIZES = [0.2] # , 0.1
EPOCHS = 100
MODES = ['lstm'] # , 'gru'
BATCH_SIZE = 64
CLASS = df_poem_filtered['label'].nunique()
MAX_LENGTH = 256

for TEST_SIZE in TEST_SIZES:
    for MODE in MODES:
        print('')
        kelas = DP1(TEST_SIZE, EPOCHS, MODE, CLASS, DATASET)
        X_train_HF, X_val_HF, y_train_HF, y_val_HF = kelas.prepareDf(df_poem_filtered)
        X_train_HF_keras, X_val_HF_keras, word_counts = kelas.kerasTokenizer(X_train_HF, X_val_HF)
        train_dataset_HF = kelas.createDataset(X_train_HF_keras, y_train_HF)
        val_dataset_HF = kelas.createDataset(X_val_HF_keras, y_val_HF)
        model = kelas.buildModel(MODE, int(word_counts//4))
        model.fit(
            train_dataset_HF,
            epochs = EPOCHS,
            batch_size = BATCH_SIZE,
            validation_data = val_dataset_HF,
            callbacks=[kelas.checkpointCallback(), kelas.earlyStopping()]
        )
        best_model = load_model(f'./model/best_model_{DATASET}_{TEST_SIZE}_{MODE}.keras')
        result = best_model.predict(X_val_HF_keras)
        predicted_labels = np.argmax(result, axis=1)
        accuracy = accuracy_score(y_val_HF, predicted_labels)
        print(f'Akurasi untuk skema {DATASET}, split test {TEST_SIZE*100}%, model {MODE} adalah {accuracy*100}%')
        with open('4.1. log_testing_result.txt', 'a') as f:
            f.write(f'\nAkurasi untuk skema {DATASET}, split test {TEST_SIZE*100}%, model {MODE} adalah {accuracy*100}% \n')

/mnt/e/Dicoding/Dicoding - Belajar Pengembangan Machine Learning/Proyek 1 - Scraping Sentiment Analysis/custom_function.py:105: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['poem'] = df['poem'].apply(self.normalizeWhitespace)
/mnt/e/Dicoding/Dicoding - Belajar Pengembangan Machine Learning/Proyek 1 - Scraping Sentiment Analysis/custom_function.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['poem'] = df['poem'].apply(self.removeOtherLanguage)
/mnt/e/Dicoding/Dicoding - Belajar Pengembangan Mach

Epoch 1/100


I0000 00:00:1742889928.401098   72883 cuda_dnn.cc:529] Loaded cuDNN version 90300


206/206 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 0.3046 - loss: 1.3769
Epoch 1: val_accuracy improved from -inf to 0.42145, saving model to ./model/best_model_deepseek_clean_0.2_lstm.keras
206/206 ━━━━━━━━━━━━━━━━━━━━ 19s 75ms/step - accuracy: 0.3048 - loss: 1.3768 - val_accuracy: 0.4215 - val_loss: 1.3285
Epoch 2/100
205/206 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.4506 - loss: 1.2912
Epoch 2: val_accuracy improved from 0.42145 to 0.51109, saving model to ./model/best_model_deepseek_clean_0.2_lstm.keras
206/206 ━━━━━━━━━━━━━━━━━━━━ 16s 75ms/step - accuracy: 0.4507 - loss: 1.2909 - val_accuracy: 0.5111 - val_loss: 1.1908
Epoch 3/100
206/206 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 0.5478 - loss: 1.0958
Epoch 3: val_accuracy improved from 0.51109 to 0.57672, saving model to ./model/best_model_deepseek_clean_0.2_lstm.keras
206/206 ━━━━━━━━━━━━━━━━━━━━ 17s 83ms/step - accuracy: 0.5479 - loss: 1.0956 - val_accuracy: 0.5767 - val_loss: 1.0486
Epoch 4/100
206/206 ━━━━━━━━━━━